### Import Library dan CSV

In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
import string
import matplotlib.pyplot as plt
import time

from indoNLP.preprocessing import replace_word_elongation, replace_slang, remove_stopwords

In [2]:
angerData = pd.read_csv('./data/AngerData.csv', delimiter='\t')
fearData = pd.read_csv('./data/FearData.csv', delimiter='\t')
joyData = pd.read_csv('./data/JoyData.csv', delimiter='\t')
loveData = pd.read_csv('./data/LoveData.csv', delimiter='\t')
sadData = pd.read_csv('./data/SadData.csv', delimiter='\t')\
    
#gabungkan data
df = pd.concat([angerData, fearData, joyData, loveData, sadData], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True) #shuffle data

df.head()

,Tweet,Label
0,"aib? bodoh benar! sebelum kata aib itu muncul,...",Anger
1,"lo takut ga sih anjir selasa depan anjir, trea...",Fear
2,"5 feb, hari ini flight balik jakarta. 12.50. s...",Sad
3,"gajian aja belum, udah ada yang ngomong ""nanti...",Sad
4,"selamat pgi jga pak suryo, salam sehat penuh s...",Joy


In [3]:
# Mengubag nama label menjadi bahasa Indonesia
label_mapping = {
    'Anger': 'Marah',
    'Fear': 'Takut',
    'Joy': 'Gembira',
    'Love': 'Cinta',
    'Sad': 'Sedih'
}
df['Label'] = df['Label'].map(label_mapping)
df.head()

,Tweet,Label
0,"aib? bodoh benar! sebelum kata aib itu muncul,...",Marah
1,"lo takut ga sih anjir selasa depan anjir, trea...",Takut
2,"5 feb, hari ini flight balik jakarta. 12.50. s...",Sedih
3,"gajian aja belum, udah ada yang ngomong ""nanti...",Sedih
4,"selamat pgi jga pak suryo, salam sehat penuh s...",Gembira


### Cleaning Data

In [4]:
def cleanDataframe(df):
    print('Missing Values:', df.isnull().sum())
    print('Duplicates:', df.duplicated().sum())

    df_cleaned = df.dropna(subset='Tweet')
    df_cleaned = df_cleaned.drop_duplicates(keep='first')

    print('\nMissing values after cleaning:', df_cleaned.isnull().sum())
    print('Duplicates after cleaning:', df_cleaned.duplicated().sum())
    return df_cleaned


kamusCleaning = pd.read_csv('kamus.csv', encoding='utf-8',
                           delimiter=';', header=None, names=['slang', 'formal'])
cleaningDict = dict(zip(kamusCleaning.slang, kamusCleaning.formal))


def applyCleaningDict(text):
    # Membersihkan kata yang tidak terbersihkan dari kamus IndoNLP dari kamus tambahan
    words = text.split()
    cleaned_words = [cleaningDict[word] if word in cleaningDict else word for word in words]
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text

def cleaningText(text):
    # Membersihkan tanda tanda sisa medsos
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Menghapus Mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Menghapus Hashtag
    text = re.sub(r'RT[\s]', '', text)  # menghapus RT
    text = re.sub(r"http\S+", '', text)  # menghapus link

    # Pembersihan Karakter
    
    text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))# Mengganti tanda baca dengan spasi (alih-alih menghapus)
    text = re.sub(r'\d+', '', text)  # Menghapus angka
    text = text.replace('\n', ' ')  # Mengganti garis baru dengan spasi

    # Normalisasi
    text = text.strip(' ')  # Menghapus karakter spasi dari kiri dan kanan teks
    text = text.lower()  # mengubah semua karakter dalam teks menjadi huruf kecil
    text = re.sub(r'(.)\1+', r'\1\1', text) # Menghapus text elongation (library indoNLP)
    text = replace_word_elongation(text) # Menghapus huruf berlebih di belakang
    text = replace_slang(text)  # Menghapus slangwords dari kamus IndoNLP
    text = applyCleaningDict(text) # Membersihkan kata yang tidak terbersihkan dari kamus IndoNLP dari kamus tambahan
    text = re.sub(r'\s+', ' ', text) # Mengganti multiple spasi dengan satu spasi
    
    return text

In [5]:
df_cleaned = cleanDataframe(df)

Missing Values: Tweet    0
Label    0
dtype: int64
Duplicates: 42

Missing values after cleaning: Tweet    0
Label    0
dtype: int64
Duplicates after cleaning: 0


In [6]:
df_cleaned['Tweet'] = df_cleaned['Tweet'].apply(cleaningText)
df_cleaned

,Tweet,Label
0,aib bodoh benar sebelum kata aib itu muncul te...,Marah
1,lo takut enggak sih anjir selasa depan anjir t...,Takut
2,feb hari ini flight balik jakarta sedih degdeg...,Sedih
3,gajian saja belum sudah ada yang mengomong nan...,Sedih
4,selamat pagi juga pak suryo salam sehat penuh ...,Gembira
...,...,...
4989,kaget banget lagi mam mie terus menonton al of...,Takut
4990,tahi banget weh ini sudah r mbak mas baru meng...,Marah
4991,sekalinya dapat au yang gua suka alur ceritany...,Cinta
4992,tapi child enak banget sih pas sama suara mark...,Gembira


In [7]:
print(df_cleaned['Label'].value_counts())

Label
Gembira    1254
Marah      1043
Sedih       995
Takut       906
Cinta       754
Name: count, dtype: int64


### Export Menjadi CSV Baru

In [8]:
# Export
df_cleaned.to_csv('./data/data_cleaned.csv', index=False)